In [2]:

from pyspark.sql.functions import count, date_sub, current_date, split, lit, when
import pandas as pd
import shutil
from decimal import Decimal
from lib import common_functions
from lib import configuration
#from lib.predict_corners import get_corner_predictions

from my_predictive_model.src.predict import get_corner_predictions

In [3]:
spark = common_functions.get_spark_session('corners')
spark.active()

In [31]:
df_corners = spark.read.option("header", True).load(f'{configuration.football_corners_input_path}/*.csv', format='csv')

df_corners = df_corners.withColumn('HomeAway', when(df_corners['Fixture'].contains('- AW'), 'AWAY').otherwise('HOME'))

df_corners = df_corners.withColumn('HomeTeam', split(df_corners['Fixture'], ' vs ')[0]) \
                       .withColumn('AwayTeam', split(df_corners['Fixture'], ' vs ')[1])


#df_corners.show(5)
df_corners.count()

1445

Filter to make new DF's

In [32]:
df_corners_history = df_corners.select(df_corners["*"]).where(    (df_corners['1HC_Res'].isNotNull())    )
corners_history_ct = df_corners_history.count()
print(corners_history_ct)

1389


In [33]:
df_corners_history_aw = df_corners_history.select(df_corners_history["*"]).where(    (df_corners_history['Fixture'].contains('- AW'))    )
df_corners_history_aw_ct = df_corners_history_aw.count()
print(df_corners_history_aw_ct)

187


In [34]:
df_corners_predict = df_corners.select(df_corners["*"]).where(    (df_corners['1AC'].isNotNull()) & (df_corners['1AC_Res'].isNull())    )
corners_predict_ct = df_corners_predict.count()
print(corners_predict_ct)

5


## Calculate the count of rows where the specified columns are equal to 0

In [35]:
count_1HC_Res_0 = df_corners_history.filter(df_corners_history['1HC_Res'] == 0).count()
count_1AC_Res_0 = df_corners_history.filter(df_corners_history['1AC_Res'] == 0).count()
count_2HC_Res_0 = df_corners_history.filter(df_corners_history['2HC_Res'] == 0).count()
count_2AC_Res_0 = df_corners_history.filter(df_corners_history['2AC_Res'] == 0).count()
count_1HC_1AC_Res_0 = df_corners_history.filter((df_corners_history['1HC_Res'] == 0) & (df_corners_history['1AC_Res'] == 0)).count()
count_2HC_2AC_Res_0 = df_corners_history.filter((df_corners_history['2HC_Res'] == 0) & (df_corners_history['2AC_Res'] == 0)).count()
df_corners_history_filter_sum_less_3 = df_corners_history.filter((df_corners_history['1HC_Res'] + df_corners_history['1AC_Res']) < 3).count()
df_corners_history_filter_sum_less_4 = df_corners_history.filter((df_corners_history['2HC_Res'] + df_corners_history['2AC_Res']) < 4).count()


# Calculate the percentage
perc_1HC_Res_0 = (count_1HC_Res_0 / corners_history_ct) * 100
perc_1AC_Res_0 = (count_1AC_Res_0 / corners_history_ct) * 100
perc_2HC_Res_0 = (count_2HC_Res_0 / corners_history_ct) * 100
perc_2AC_Res_0 = (count_2AC_Res_0 / corners_history_ct) * 100
perc_1HC_1AC_Res_0 = (count_1HC_1AC_Res_0 / corners_history_ct) * 100
perc_2HC_2AC_Res_0 = (count_2HC_2AC_Res_0 / corners_history_ct) * 100
perc_sum_less_3 = (df_corners_history_filter_sum_less_3 / corners_history_ct) * 100
perc_sum_less_4 = (df_corners_history_filter_sum_less_4 / corners_history_ct) * 100


print(f'Percentage of 1HC_Res zero: {perc_1HC_Res_0}%')
print(f'Percentage of 1AC_Res zero: {perc_1AC_Res_0}%')
print(f'Percentage of 2HC_Res zero: {perc_2HC_Res_0}%')
print(f'Percentage of 2AC_Res zero: {perc_2AC_Res_0}%')
print(f'Percentage of 1HC_Res and 1AC_Res both zero: {perc_1HC_1AC_Res_0}%')
print(f'Percentage of 2HC_Res and 2AC_Res both zero: {perc_2HC_2AC_Res_0}%')
print(f'Percentage of rows where 1HC_Res + 1AC_Res < 3: {perc_sum_less_3}%')
print(f'Percentage of rows where 1HC_Res + 1AC_Res < 4: {perc_sum_less_4}%')

Percentage of 1HC_Res zero: 9.215262778977682%
Percentage of 1AC_Res zero: 15.766738660907128%
Percentage of 2HC_Res zero: 8.423326133909287%
Percentage of 2AC_Res zero: 13.390928725701945%
Percentage of 1HC_Res and 1AC_Res both zero: 0.7199424046076314%
Percentage of 2HC_Res and 2AC_Res both zero: 1.0079193664506838%
Percentage of rows where 1HC_Res + 1AC_Res < 3: 14.902807775377969%
Percentage of rows where 1HC_Res + 1AC_Res < 4: 24.406047516198704%


## Get Past Month History for re-use.

In [36]:
# Filter data from the previous month
df_corners_history_prev_month = df_corners_history.filter(df_corners_history['InsertDate'] >= date_sub(current_date(), 15))
df_corners_history_prev_month_ct = df_corners_history_prev_month.count()
print(df_corners_history_prev_month_ct)

125


In [37]:
# Select the relevant columns for home and away DataFrames
df_corners_history_home = df_corners_history_prev_month.select('HomeTeam','1HC','1AC','2HC','2AC','1HCA','1ACA','2HCA','2ACA')
df_corners_history_away = df_corners_history_prev_month.select('AwayTeam','1HC','1AC','2HC','2AC','1HCA','1ACA','2HCA','2ACA')

# Replace 1AC contents with '-'
df_corners_history_home = df_corners_history_home.withColumn('1AC', lit('-'))
df_corners_history_home = df_corners_history_home.withColumn('2AC', lit('-'))
df_corners_history_home = df_corners_history_home.withColumn('1ACA', lit('-'))
df_corners_history_home = df_corners_history_home.withColumn('2ACA', lit('-'))

df_corners_history_away = df_corners_history_away.withColumn('1HC', lit('-'))
df_corners_history_away = df_corners_history_away.withColumn('2HC', lit('-'))
df_corners_history_away = df_corners_history_away.withColumn('1HCA', lit('-'))
df_corners_history_away = df_corners_history_away.withColumn('2HCA', lit('-'))

## Train and export predictions using sklearn

In [38]:
get_corner_predictions(df_corners_history.toPandas(),df_corners_predict.toPandas())

Prediction dataset with results saved successfully!


In [39]:
df_corners_history_more_two = df_corners_history.select(df_corners["*"]).where(    (df_corners['WinOverTwo'] == 1)    )
corners_history_more_two_ct = df_corners_history_more_two.count()
print(corners_history_more_two_ct)
print(corners_history_more_two_ct/corners_history_ct*100)

240
17.278617710583152


In [40]:
avg_cols = {'1HC': 'avg', '1AC': 'avg', '2HC': 'avg', '2AC': 'avg', '1HCA': 'avg', '1ACA': 'avg', '2HCA': 'avg', '2ACA': 'avg', 'PastForTotal': 'avg', 'PastAgainstTotal': 'avg' }

df_corners_history_more_two_avg = df_corners_history_more_two.agg(avg_cols)
df_c2 = df_corners_history_more_two_avg

#df_corners_history_more_two_avg.select(df_c2['avg(1HC)'],df_c2['avg(1AC)'],df_c2['avg(2HC)'],df_c2['avg(2AC)'],df_c2['avg(1HCA)'],df_c2['avg(1ACA)'],df_c2['avg(2HCA)'],df_c2['avg(2ACA)'],df_c2['avg(PastForTotal)'],df_c2['avg(PastAgainstTotal)']).show()

## Predict over averages

In [41]:
row = df_corners_history_more_two_avg.first()
perc = 0.9

avg_1HC = Decimal(row['avg(1HC)'] * perc).quantize(Decimal("0.00")) 
avg_1AC = Decimal(row['avg(1AC)'] * perc).quantize(Decimal("0.00"))
avg_2HC = Decimal(row['avg(2HC)'] * perc).quantize(Decimal("0.00"))
avg_2AC = Decimal(row['avg(2AC)'] * perc).quantize(Decimal("0.00"))
avg_1HCA = Decimal(row['avg(1HCA)'] * perc).quantize(Decimal("0.00"))
avg_1ACA = Decimal(row['avg(1ACA)'] * perc).quantize(Decimal("0.00"))
avg_2HCA = Decimal(row['avg(2HCA)'] * perc).quantize(Decimal("0.00"))
avg_2ACA = Decimal(row['avg(2ACA)'] * perc).quantize(Decimal("0.00"))
avg_PastForTotal = Decimal(row['avg(PastForTotal)'] * perc).quantize(Decimal("0.00"))
avg_PastAgainstTotal = Decimal(row['avg(PastAgainstTotal)'] * perc).quantize(Decimal("0.00"))

print(f'avg_1HC - {avg_1HC} / avg_1AC - {avg_1AC} / avg_2HC - {avg_2HC} / avg_2AC - {avg_2AC} / avg_PastForTotal - {avg_PastForTotal} / avg_PastAgainstTotal - {avg_PastAgainstTotal}')

print(f'avg_1HCA - {avg_1HCA} / avg_1ACA - {avg_1ACA} / avg_2HCA - {avg_2HCA} / avg_2ACA - {avg_2ACA}')

avg_1HC - 2.57 / avg_1AC - 2.20 / avg_2HC - 2.75 / avg_2AC - 2.43 / avg_PastForTotal - 9.95 / avg_PastAgainstTotal - 9.59
avg_1HCA - 2.01 / avg_1ACA - 2.48 / avg_2HCA - 2.31 / avg_2ACA - 2.79


In [42]:
# (df_corners_history['1HCA'] > avg_1HCA) & (df_corners_history['1ACA'] > avg_1ACA) & (df_corners_history['2HCA'] > avg_2HCA) & (df_corners_history['2ACA'] > avg_2ACA)

df_pick_over_pred = df_corners_predict.select(df_corners_history["*"]).where(    (df_corners_history['1HC'] >= avg_1HC) & (df_corners_history['1AC'] >= avg_1AC) & (df_corners_history['2HC'] >= avg_2HC) & (df_corners_history['2AC'] >= avg_2AC)    )
df_pick_over_pred_ct = df_pick_over_pred.count()
print(df_pick_over_pred_ct)
df_pick_over_pred.show()

1
+-----+--------------------+----+----+----+----+-------+-------+-------+-------+----+----+----+----+----------+----------+------------+-------------+-------------+-------------+------------+----------------+----------------+-----------------+-----------+----------------+-----------------+---------------+----------------+--------------+----------------+---------------------+------------------+------------+-------------+-----------+------------+----------+----------+-----------+-----------+--------+-------------+----------+
|   ID|             Fixture| 1HC| 1AC| 2HC| 2AC|1HC_Res|1AC_Res|2HC_Res|2AC_Res|1HCA|1ACA|2HCA|2ACA|windrawwin|InsertDate|PastForFirst|PastForSecond|PastHomeTotal|PastAwayTotal|PastForTotal|PastAgainstTotal|ResultTotalFirst|ResultTotalSecond|ResultTotal|HistoryOverFirst|HistoryOverSecond|HistoryOverBoth|HistoryOverTotal|HistoryOverTwo|HistoryOverThree|HistoryAgainstOverTwo|HistoryOverTwoBoth|WinOverFirst|WinOverSecond|WinOverBoth|WinOverTotal|WinOverOne|WinOverTwo|W

## Get historic teams with highest averages

In [43]:
df_corners_history.createOrReplaceTempView("corners_split")

In [44]:
spark.sql(
    "SELECT HomeTeam, COUNT(*) AS CT, avg(1HC_Res) AS 1HC_Res, avg(2HC_Res) AS 2HC_Res, \
    avg(1HC_Res) + avg(2HC_Res) AS Tot, \
    avg(1HC) AS 1HC, avg(2HC) AS 2HC, avg(1HCA) AS 1HCA, avg(2HCA) AS 2HCA, \
    sum(WinOverFirst)/COUNT(*)*100 AS WinOverFirst,sum(WinOverBoth)/COUNT(*)*100 AS WinOverBoth,sum(WinOverTotal)/COUNT(*)*100 AS WinOverTotal, \
    sum(WinOverOne)/COUNT(*)*100 AS WinOverOne,sum(WinOverTwo)/COUNT(*)*100 AS WinOverTwo \
    FROM corners_split \
    WHERE ID LIKE '%,%' AND HomeTeam = 'Telstar' \
    GROUP BY HomeTeam \
    ORDER BY Tot DESC"
    ).show(10)

+--------+---+-------+-------+---+------------------+-----+----+-----+------------+-----------+------------+----------+----------+
|HomeTeam| CT|1HC_Res|2HC_Res|Tot|               1HC|  2HC|1HCA| 2HCA|WinOverFirst|WinOverBoth|WinOverTotal|WinOverOne|WinOverTwo|
+--------+---+-------+-------+---+------------------+-----+----+-----+------------+-----------+------------+----------+----------+
| Telstar|  2|    5.0|    3.5|8.5|3.4699999999999998|3.085| 1.3|2.005|        50.0|       50.0|       100.0|      50.0|       0.0|
+--------+---+-------+-------+---+------------------+-----+----+-----+------------+-----------+------------+----------+----------+



In [45]:
spark.sql(
    "SELECT AwayTeam, COUNT(*) AS CT, avg(1AC_Res) AS 1AC_Res, avg(2AC_Res) AS 2AC_Res, \
    avg(1AC_Res) + avg(2AC_Res) AS Tot, \
    avg(1AC) AS 1AC, avg(2AC) AS 2AC, avg(1ACA) AS 1ACA, avg(2ACA) AS 2ACA, \
    sum(WinOverFirst)/COUNT(*)*100 AS WinOverFirst,sum(WinOverBoth)/COUNT(*)*100 AS WinOverBoth,sum(WinOverTotal)/COUNT(*)*100 AS WinOverTotal, \
    sum(WinOverOne)/COUNT(*)*100 AS WinOverOne,sum(WinOverTwo)/COUNT(*)*100 AS WinOverTwo \
    FROM corners_split \
    WHERE ID LIKE '%,%' AND AwayTeam = 'Excelsior' \
    GROUP BY AwayTeam \
    ORDER BY Tot DESC"
    ).show(10)

+---------+---+-------+-------+---+----+----+----+----+------------+-----------+------------+----------+----------+
| AwayTeam| CT|1AC_Res|2AC_Res|Tot| 1AC| 2AC|1ACA|2ACA|WinOverFirst|WinOverBoth|WinOverTotal|WinOverOne|WinOverTwo|
+---------+---+-------+-------+---+----+----+----+----+------------+-----------+------------+----------+----------+
|Excelsior|  1|    1.0|    4.0|5.0|2.38|3.85|2.69|1.92|         0.0|        0.0|       100.0|     100.0|       0.0|
+---------+---+-------+-------+---+----+----+----+----+------------+-----------+------------+----------+----------+



In [46]:
games_count = 3

In [47]:
df_corners_history_avg_home = spark.sql(
    f"SELECT HomeTeam, COUNT(*) AS CT, avg(1HC_Res) AS 1HC_Res, avg(2HC_Res) AS 2HC_Res, \
    avg(1HC_Res) + avg(2HC_Res) AS Tot, \
    avg(1HC) AS 1HC, avg(2HC) AS 2HC, avg(1HCA) AS 1HCA, avg(2HCA) AS 2HCA, \
    sum(WinOverFirst)/COUNT(*)*100 AS WinOverFirst,sum(WinOverBoth)/COUNT(*)*100 AS WinOverBoth,sum(WinOverTotal)/COUNT(*)*100 AS WinOverTotal, \
    sum(WinOverOne)/COUNT(*)*100 AS WinOverOne,sum(WinOverTwo)/COUNT(*)*100 AS WinOverTwo \
    FROM corners_split \
    WHERE ID LIKE '%,%' \
    GROUP BY HomeTeam \
    HAVING CT > {games_count} \
    ORDER BY Tot DESC"
    )#.show(20)

In [48]:
df_corners_history_avg_away_fav = spark.sql(
    f"SELECT AwayTeam, COUNT(*) AS CT, avg(1AC_Res) AS 1AC_Res, avg(2AC_Res) AS 2AC_Res, \
    avg(1AC_Res) + avg(2AC_Res) AS Tot, \
    avg(1AC) AS 1AC, avg(2AC) AS 2AC, avg(1ACA) AS 1ACA, avg(2ACA) AS 2ACA, \
    sum(WinOverFirst)/COUNT(*)*100 AS WinOverFirst,sum(WinOverBoth)/COUNT(*)*100 AS WinOverBoth,sum(WinOverTotal)/COUNT(*)*100 AS WinOverTotal, \
    sum(WinOverOne)/COUNT(*)*100 AS WinOverOne,sum(WinOverTwo)/COUNT(*)*100 AS WinOverTwo \
    FROM corners_split \
    WHERE ID LIKE '%,%' AND FIXTURE LIKE '%- AW' \
    GROUP BY AwayTeam \
    HAVING CT > 2 \
    ORDER BY Tot DESC"
    )#.show(20)

In [49]:
df_corners_history_avg_away = spark.sql(
    f"SELECT AwayTeam, COUNT(*) AS CT, avg(1AC_Res) AS 1AC_Res, avg(2AC_Res) AS 2AC_Res, \
    avg(1AC_Res) + avg(2AC_Res) AS Tot, \
    avg(1AC) AS 1AC, avg(2AC) AS 2AC, avg(1ACA) AS 1ACA, avg(2ACA) AS 2ACA, \
    sum(WinOverFirst)/COUNT(*)*100 AS WinOverFirst,sum(WinOverBoth)/COUNT(*)*100 AS WinOverBoth,sum(WinOverTotal)/COUNT(*)*100 AS WinOverTotal, \
    sum(WinOverOne)/COUNT(*)*100 AS WinOverOne,sum(WinOverTwo)/COUNT(*)*100 AS WinOverTwo \
    FROM corners_split \
    WHERE ID LIKE '%,%' \
    GROUP BY AwayTeam \
    HAVING CT > {games_count} \
    ORDER BY Tot DESC"
    )#.show(20)

## Lookup predictions against past high avgs

In [50]:
df_intersect_home = df_corners_predict.join(df_corners_history_avg_home, df_corners_predict.HomeTeam == df_corners_history_avg_home.HomeTeam, 'inner')
df_intersect_away_fav = df_corners_predict.join(df_corners_history_avg_away_fav, df_corners_predict.AwayTeam == df_corners_history_avg_away_fav.AwayTeam, 'inner')
df_intersect_away = df_corners_predict.join(df_corners_history_avg_away, df_corners_predict.AwayTeam == df_corners_history_avg_away.AwayTeam, 'inner')

df_intersect = df_intersect_home.union(df_intersect_away_fav).union(df_intersect_away)
df_intersect.show()

+-----+--------------------+----+----+----+----+-------+-------+-------+-------+----+----+----+----+----------+----------+------------+-------------+-------------+-------------+------------+----------------+----------------+-----------------+-----------+----------------+-----------------+---------------+----------------+--------------+----------------+---------------------+------------------+------------+-------------+-----------+------------+----------+----------+-----------+-----------+--------+-------------+----------+-------------+---+------------------+------------------+-----------------+------------------+------------------+-----+------------------+-----------------+-----------------+-----------------+-----------------+-----------------+
|   ID|             Fixture| 1HC| 1AC| 2HC| 2AC|1HC_Res|1AC_Res|2HC_Res|2AC_Res|1HCA|1ACA|2HCA|2ACA|windrawwin|InsertDate|PastForFirst|PastForSecond|PastHomeTotal|PastAwayTotal|PastForTotal|PastAgainstTotal|ResultTotalFirst|ResultTotalSecond|Res

## Write to excel

In [51]:
with pd.ExcelWriter(f'{configuration.football_corners_output_path}/corners.xlsx') as writer:  
     
     df_corners_history_avg_home.toPandas().to_excel(writer, sheet_name=f'df_corners_history_avg_home', index=False)
     df_corners_history_avg_away_fav.toPandas().to_excel(writer, sheet_name=f'df_corners_history_avg_away_fav', index=False)
     df_corners_history_avg_away.toPandas().to_excel(writer, sheet_name=f'df_corners_history_avg_away', index=False)
     df_intersect.toPandas().to_excel(writer, sheet_name=f'df_intersect', index=False)
     
     df_corners_history_more_two_avg.toPandas().to_excel(writer, sheet_name=f'over_two_avg', index=False)
     df_pick_over_pred.toPandas().to_excel(writer, sheet_name=f'pick_over_pred - {perc}%', index=False)
     df_corners_history_more_two.toPandas().to_excel(writer, sheet_name=f'history_more_two', index=False)

     #df_corners_history_home.toPandas().to_excel(writer, sheet_name=f'df_corners_history_home', index=False)
     #df_corners_history_away.toPandas().to_excel(writer, sheet_name=f'df_corners_history_away', index=False) 


## Make Picks from - df_corners_predict

In [52]:
df_temp_table = df_corners_history.select(df_corners["*"]).where(    (df_corners['windrawwin'].isNotNull())   )

In [53]:
df_temp_table.createOrReplaceTempView("corners")
df_corners_predict.createOrReplaceTempView("corners_predict")

In [54]:
what_to_predict = 'WinOverFirst'

In [55]:
result = spark.sql(f"SELECT avg(1HC) as avg_1HC, avg(1AC) as avg_1AC, avg(2HC) as avg_2HC, avg(2AC) as avg_2AC, avg(1HCA) as avg_1HCA, avg(1ACA) as avg_1ACA, avg(2HCA) as avg_2HCA, avg(2ACA) as avg_2ACA, avg(PastForTotal) as avg_PastForTotal, avg(PastAgainstTotal) as avg_PastAgainstTotal, SUM({what_to_predict})/count(*)*100 AS PERC FROM corners").first()

avg_1HC = Decimal(result['avg_1HC']).quantize(Decimal("0.00"))
avg_1AC = Decimal(result['avg_1AC']).quantize(Decimal("0.00"))
avg_2HC = Decimal(result['avg_2HC']).quantize(Decimal("0.00"))
avg_2AC = Decimal(result['avg_2AC']).quantize(Decimal("0.00"))
avg_1HCA = Decimal(result['avg_1HCA']).quantize(Decimal("0.00"))
avg_1ACA = Decimal(result['avg_1ACA']).quantize(Decimal("0.00"))
avg_2HCA = Decimal(result['avg_2HCA']).quantize(Decimal("0.00"))
avg_2ACA = Decimal(result['avg_2ACA']).quantize(Decimal("0.00"))
avg_PastForTotal = Decimal(result['avg_PastForTotal']).quantize(Decimal("0.00"))
avg_PastAgainstTotal = Decimal(result['avg_PastAgainstTotal']).quantize(Decimal("0.00"))

print(f'avg_1HC: {avg_1HC}, avg_1AC: {avg_1AC}, avg_2HC: {avg_2HC}, avg_2AC: {avg_2AC}')
print(f'avg_1HCA: {avg_1HCA}, avg_1ACA: {avg_1ACA}, avg_2HCA: {avg_2HCA}, avg_2ACA: {avg_2ACA}')
print(f'avg_PastForTotal: {avg_PastForTotal}, avg_PastAgainstTotal: {avg_PastAgainstTotal}')

avg_1HC: 2.77, avg_1AC: 2.25, avg_2HC: 3.08, avg_2AC: 2.56
avg_1HCA: 2.21, avg_1ACA: 2.67, avg_2HCA: 2.47, avg_2ACA: 3.02
avg_PastForTotal: 10.66, avg_PastAgainstTotal: 10.37


In [56]:
spark.sql(f"SELECT HomeAway,windrawwin,SUM({what_to_predict}),count(*) AS TOT,SUM({what_to_predict})/count(*)*100 AS PERC FROM corners GROUP BY HomeAway,windrawwin HAVING PERC > 85 AND TOT > 10 ORDER BY PERC DESC").show()

+--------+----------+-----------------+---+----+
|HomeAway|windrawwin|sum(WinOverFirst)|TOT|PERC|
+--------+----------+-----------------+---+----+
+--------+----------+-----------------+---+----+



In [57]:
spark.sql(f"SELECT * FROM corners_predict WHERE windrawwin IN ('2-2') AND HomeAway = 'AWAY' AND 1HC >= {avg_1HC}*0.8 AND 1AC >= {avg_1AC}*0.8").show(50)

+-----+--------------------+----+----+----+----+-------+-------+-------+-------+----+----+----+----+----------+----------+------------+-------------+-------------+-------------+------------+----------------+----------------+-----------------+-----------+----------------+-----------------+---------------+----------------+--------------+----------------+---------------------+------------------+------------+-------------+-----------+------------+----------+----------+-----------+-----------+--------+--------+--------------+
|   ID|             Fixture| 1HC| 1AC| 2HC| 2AC|1HC_Res|1AC_Res|2HC_Res|2AC_Res|1HCA|1ACA|2HCA|2ACA|windrawwin|InsertDate|PastForFirst|PastForSecond|PastHomeTotal|PastAwayTotal|PastForTotal|PastAgainstTotal|ResultTotalFirst|ResultTotalSecond|ResultTotal|HistoryOverFirst|HistoryOverSecond|HistoryOverBoth|HistoryOverTotal|HistoryOverTwo|HistoryOverThree|HistoryAgainstOverTwo|HistoryOverTwoBoth|WinOverFirst|WinOverSecond|WinOverBoth|WinOverTotal|WinOverOne|WinOverTwo|WinM